# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-12.31,95,100,3.85,RU,2025-03-06 02:45:36
1,1,tofol,5.3248,163.0078,28.23,71,100,10.19,FM,2025-03-06 02:45:38
2,2,albany,42.6001,-73.9662,10.36,93,100,1.79,US,2025-03-06 02:45:39
3,3,bethel,41.3712,-73.4140,10.36,91,100,1.34,US,2025-03-06 02:45:40
4,4,vorgashor,67.5833,63.9500,-20.94,100,93,2.28,RU,2025-03-06 02:45:18


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

humidity_range = city_data_df['Humidity'].max() - city_data_df['Humidity'].min()
scale_value = max(0.1, min(1, humidity_range / 100))  # Normalize scaling factor

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=scale_value,  # Dynamically adjust scale
    color="City", 
    alpha=0.5, 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500
)


# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Lat"].notna()) & (city_data_df["Lng"].notna())]  

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-12.31,95,100,3.85,RU,2025-03-06 02:45:36
1,1,tofol,5.3248,163.0078,28.23,71,100,10.19,FM,2025-03-06 02:45:38
2,2,albany,42.6001,-73.9662,10.36,93,100,1.79,US,2025-03-06 02:45:39
3,3,bethel,41.3712,-73.4140,10.36,91,100,1.34,US,2025-03-06 02:45:40
4,4,vorgashor,67.5833,63.9500,-20.94,100,93,2.28,RU,2025-03-06 02:45:18


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,talnakh,RU,69.4865,88.3972,95,
1,tofol,FM,5.3248,163.0078,71,
2,albany,US,42.6001,-73.9662,93,
3,bethel,US,41.3712,-73.4140,91,
4,vorgashor,RU,67.5833,63.9500,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 5000  
params = {
    'apiKey': geoapify_key,  
    'categories': 'accommodation.hotel',  
    'limit': 1,  
    'radius': radius,  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lng

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)
        name_address = response.json()  

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and len(name_address["features"]) > 0:
            hotel_name = name_address["features"][0]["properties"]["name"]
        else:
            hotel_name = "No hotel found"
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    except (requests.exceptions.RequestException, KeyError, IndexError):
        # If there is an error, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
talnakh - nearest hotel: Талнах
tofol - nearest hotel: Nautilus Resort
albany - nearest hotel: Homewood Suites
bethel - nearest hotel: Hampton Inn Danbury
vorgashor - nearest hotel: София
port-aux-francais - nearest hotel: Keravel
yellowknife - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
grytviken - nearest hotel: Shorty's Motel
lata - nearest hotel: Residencial Botânico
hithadhoo - nearest hotel: Pebbles Inn
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
bilibino - nearest hotel: No hotel found
ust-nera - nearest hotel: гостиница "Солнечная"
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
dudinka - nearest hotel: Виктория
iqaluit - nearest hotel: Frobisher Inn
punta arenas - nearest hotel: Hotel Lacolet
keflavik - nearest hotel: Núpan Deluxe
puerto natales - nearest hotel: Dorotea Patagonia Hostel
waitangi 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,talnakh,RU,69.4865,88.3972,95,Талнах
1,tofol,FM,5.3248,163.0078,71,Nautilus Resort
2,albany,US,42.6001,-73.9662,93,Homewood Suites
3,bethel,US,41.3712,-73.4140,91,Hampton Inn Danbury
4,vorgashor,RU,67.5833,63.9500,100,София


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Calculate the range of humidity and scale value dynamically
humidity_range = hotel_df['Humidity'].max() - hotel_df['Humidity'].min()
scale_value = max(0.1, min(1, humidity_range / 100))  # Normalize scaling factor

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",  
    "Lat",
    geo=True, 
    size="Humidity",  
    scale=scale_value, 
    color="Hotel Name",  
    alpha=0.5,  
    tiles="OSM", 
    frame_width=700,  
    frame_height=500,  
    hover_cols=["City", "Hotel Name", "Humidity"] 
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City)